# Preprocess M600


Notebook que preprocesa las imágenes, cambia su tamaño, las normaliza y las organiza para Keras.

In [ ]:
import os
import pandas as pd
import PIL
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm
import shutil
import random

version=(PIL.__version__)
#Función para analizar las imagenes de varios directorios
display(version)


import matplotlib.pyplot as plt
from tqdm import tqdm



## Analisis, preprocesamiento y organizacion

In [ ]:

def Analisis(directorios, grano):
    df=pd.DataFrame()
    print(grano)
    for element in tqdm(directorios):
        # Directorio donde se encuentran las imágenes

        directorio_imagenes = r"../../grainspace/G600/" + grano + r"/" + element
        #directorio_imagenes = r"D:/Data_Science/grainspace/G600/" + grano + r"/" + element

        # Lista para almacenar las dimensiones de las imágenes
        dimensiones = []

        no_image=0
        image_count = 0
        # Recorre todos los archivos en el directorio de imágenes
        try:
            for archivo in tqdm(os.listdir(directorio_imagenes)):
                # Asegúrate de que el archivo es una imagen
                if archivo.endswith('.jpg') or archivo.endswith('.png'):
                    # Abre la imagen
                    imagen = Image.open(os.path.join(directorio_imagenes, archivo))
                    # Obtiene las dimensiones de la imagen
                    ancho, alto = imagen.size
                    # Añade las dimensiones a la lista
                    dimensiones.append({'nombre': archivo, 'ancho': ancho, 'alto': alto})
        except:
            print("no existe el directorio ",os.path.join(directorio_imagenes, archivo))
            

        # Crea un DataFrame de pandas a partir de la lista de dimensiones
        df_temp =pd.DataFrame(dimensiones)
        df=pd.concat([df,df_temp])
    # Guarda el DataFrame como un archivo CSV
    df.to_csv('dimensiones_imagenes.csv', index=False)
    print(df.head(5))
    print(df.describe(),"\n")
    print("Los valores maximos son:\n",df.max(),"\n Los minimos son:\n", df.min())

    


las imagenes analizadas de diferentes tamaños seran usadas en un modelo de ML, se debe cambiar el tamaño de cada una y normalizar.
Para ello se va a usar la biblioteca Pillow para el cambio de tamaño y OpenCV para la normalización.
A la vez se guardarn en una proporción 75:25 en carpetas de entrenamiento y validación para el uso de keras.

In [ ]:

#Fucnión para normalizar y cambiar el tamaño de las imagenes. Tambien intentara organizar las imagenes para keras
def PreprocessOrganize(directorios,grano):
    df2=pd.DataFrame()
    print(grano)
    for element in tqdm(directorios):
        # Directorio donde se encuentran las imágenes
        directorio_imagenes = r"../../grainspace/G600/" + grano + r"/" + element

        # Tamaño deseado para las imágenes
        tamaño_deseado = (150, 150)

        # Lista para almacenar las dimensiones de las imágenes
        dimensiones = []

        # Recorre todos los archivos en el directorio de imágenes
        for archivo in tqdm(os.listdir(directorio_imagenes)):
            # Asegúrate de que el archivo es una imagen
            if archivo.endswith('.jpg') or archivo.endswith('.png'):
                # Abre la imagen
                imagen = Image.open(os.path.join(directorio_imagenes, archivo))
                # Cambia el tamaño de la imagen
                imagen = imagen.resize(tamaño_deseado)
                # Convierte la imagen a una matriz numpy
                imagen_np = np.array(imagen)
                # Normaliza la imagen al rango 0-255
                imagen_normalizada = cv2.normalize(imagen_np, None, 0, 255, cv2.NORM_MINMAX)
                # Añade las dimensiones a la lista
                dimensiones.append({'nombre': archivo, 'ancho': imagen_normalizada.shape[1], 'alto': imagen_normalizada.shape[0]})
                
                # Directorios donde se guardarán las imágenes modificadas
                directorio_train = r"../../grainspace/G600/pre/"+grano+"/train/"+element
                directorio_validation = r"../../grainspace/G600/pre/"+grano+"/validation/" +element

                directorio_train2 = r"../../grainspace/G600/pre/both/train/"+element+"_"+grano
                directorio_validation2 = r"../../grainspace/G600/pre/both/validation/" +element+"_"+grano
                
                
                # Asegúrate de que los directorios de salida existen
                try:
                    os.makedirs(directorio_train, exist_ok=True)
                    os.makedirs(directorio_validation, exist_ok=True)
                    os.makedirs(directorio_train2, exist_ok=True)
                    os.makedirs(directorio_validation2, exist_ok=True)                    
                except:
                    os.makedirs(directorio_train)
                    os.makedirs(directorio_validation)
                    os.makedirs(directorio_train2)
                    os.makedirs(directorio_validation2) 
                                   
                # Decide aleatoriamente si la imagen va a train o validation
                if random.random() < 0.75:  # 75% de probabilidad para train
                    directorio_salida = directorio_train
                    directorio_salida2 = directorio_train2
                else:  # 25% de probabilidad para validation
                    directorio_salida = directorio_validation
                    directorio_salida2= directorio_validation2
                
                # Guarda la imagen normalizada
                imagen_salida = Image.fromarray(imagen_normalizada)
                imagen_salida.save(os.path.join(directorio_salida, archivo))
                imagen_salida.save(os.path.join(directorio_salida2, archivo))

                

        # Crea un DataFrame de pandas a partir de la lista de dimensiones
        df_temp =pd.DataFrame(dimensiones)
        df2=pd.concat([df2,df_temp])
        # Guarda el DataFrame como un archivo CSV
        df2.to_csv('dimensiones_Finales.csv', index=False)
    print(df2.head(5))
    print(df2.describe(),"\n")
    print("Los valores maximos son:\n",df2.max(),"\n Los minimos son:\n", df2.min())


Lamamos a la función de analsis, luego hacemos cambios con la función de preprocesamiento y finalmente volveremos a analizar las imagenes modificadas para asegurar que hayan sido correctamente procesadas

In [ ]:
directorios=[r"\NOR", r"/AP", r"/BN", r"/FM", r"/HD", r"/MY", r"/SD"]
grano=r"maize"
print("-------------------***--------------------------- \n                 Analisis\n-------------------***---------------------------")
Analisis(directorios , grano)

print("-------------------***--------------------------- \n        Preprosesar y Organizar\n-------------------***---------------------------")
PreprocessOrganize(directorios, r"maize")
print("-------------------***--------------------------- \n                   Listo\n-------------------***---------------------------")


Ahora nos aseguramos del exito, repitiendo el primer script a las nuevas imagenes.

In [ ]:
directorio_salida=[r"\NOR", r"/AP", r"/BN", r"/FM", r"/HD", r"/MY", r"/SD"]
grano=r"/pre/maize/"
#Analisis(directorios , grano)


Ahora repetiremos para el trigo(wheat) 

In [ ]:
wheat_dir=[]
for element in directorios: wheat_dir.append(element.replace("maize","wheat"))
wheat_dir.extend(["/BP","/FS"])
wheat_dir.remove("/FM")
wheat_dir.remove("/HD")

print(wheat_dir)
print("-------------------***--------------------------- \n                 Analisis\n-------------------***---------------------------")

Analisis(wheat_dir,r"wheat")
print("-------------------***--------------------------- \n        Preprosesar y Organizar\n-------------------***---------------------------")

PreprocessOrganize(wheat_dir,r"wheat")
print("-------------------***--------------------------- \n                   Listo\n-------------------***---------------------------")